# FrozenLake之值循环(Value Iteration)

In [1]:
# 载入相关套件
import numpy as np
import gym

In [2]:
# 环境
env = gym.make('FrozenLake-v1')
env.reset()

0

In [3]:
nS = env.observation_space.n
nA = env.action_space.n

In [4]:
# 值循环函数
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    # 计算行动值函数
    def one_step_lookahead(state, V):
        A = np.zeros(nA)
        for a in range(nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A

    # 状态值函数初始化
    V = np.zeros(nS)
    while True:
        delta = 0
        # 更新每个状态值的函数
        for s in range(nS):
            # 计算下一步的行动值函数
            A = one_step_lookahead(s, V)
            best_action_value = np.max(A)
            # 比较更新前后的差值，取最大值
            delta = max(delta, np.abs(best_action_value - V[s]))
            # 更新状态值函数
            V[s] = best_action_value        
        # 若最大差值 < 门槛值，则停止评估
        if delta < theta:
            break

    # 一开始采随机策略，往上/下/左/右走的机率(π)均等
    policy = np.zeros([nS, nA])
    for s in range(nS):
        # 计算下一步的行动值函数
        A = one_step_lookahead(s, V)
        # 选择最佳行动
        best_action = np.argmax(A)
        # 永远采取最佳行动
        policy[s, best_action] = 1.0
    
    return policy, V            

In [5]:
# 执行值循环
policy, v = value_iteration(env)

In [7]:
# 显示结果
print("策略机率分配:")
print(policy)
print("")

print("4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), (int(nS ** 0.5), int(nS ** 0.5))))
print("")

print("4x4 状态值函数:")
print(v.reshape((int(nS ** 0.5), int(nS ** 0.5))))

策略机率分配:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]

4x4 状态值函数:
[[0.82182145 0.82126109 0.82087163 0.82067347]
 [0.82199325 0.         0.52824715 0.        ]
 [0.82226231 0.82260733 0.76389785 0.        ]
 [0.         0.88171208 0.94085038 0.        ]]
